### UNIFORM COST SEARCH 

In [4]:
g = {
    'A': [('B', 1), ('C', 4)],
    'B': [('C', 2), ('D', 5)],
    'C': [('D', 1)],
    'D': []
}

In [7]:
import heapq

def ucs(g, s, goal):
    pq = []                                  # priority queue
    heapq.heappush(pq, (0, s))               # (cost, node)

    dist = {s: 0}
    parent = {s: None}

    print("Starting Uniform Cost Search")
    print("Start Node:", s)
    print("Goal Node :", goal)
    print("-" * 40)

    while pq:
        cost, u = heapq.heappop(pq)

        print(f"Expanding node: {u}")
        print(f"Current cost : {cost}")

        if u == goal:
            print("\nGoal reached!")
            break

        for v, w in g[u]:
            nc = cost + w
            print(f"  Checking neighbor {v} with edge cost {w}")

            if v not in dist or nc < dist[v]:
                dist[v] = nc
                parent[v] = u
                heapq.heappush(pq, (nc, v))
                print(f"  → Updating {v}, new cost = {nc}")

        print("-" * 40)

    # Path reconstruction
    path = []
    cur = goal
    while cur:
        path.append(cur)
        cur = parent[cur]
    path.reverse()

    print("\nFinal Results")
    print("Optimal Path :", " → ".join(path))
    print("Total Cost   :", dist[goal])

    return path, dist[goal]

ucs(g, 'A', 'D')

Starting Uniform Cost Search
Start Node: A
Goal Node : D
----------------------------------------
Expanding node: A
Current cost : 0
  Checking neighbor B with edge cost 1
  → Updating B, new cost = 1
  Checking neighbor C with edge cost 4
  → Updating C, new cost = 4
----------------------------------------
Expanding node: B
Current cost : 1
  Checking neighbor C with edge cost 2
  → Updating C, new cost = 3
  Checking neighbor D with edge cost 5
  → Updating D, new cost = 6
----------------------------------------
Expanding node: C
Current cost : 3
  Checking neighbor D with edge cost 1
  → Updating D, new cost = 4
----------------------------------------
Expanding node: C
Current cost : 4
  Checking neighbor D with edge cost 1
----------------------------------------
Expanding node: D
Current cost : 4

Goal reached!

Final Results
Optimal Path : A → B → C → D
Total Cost   : 4


(['A', 'B', 'C', 'D'], 4)

In [16]:
from collections import deque

def bfs_unweighted(g, s, goal):
    q = deque([s])
    vis = {s}
    parent = {s: None}

    while q:
        u = q.popleft()

        if u == goal:
            break

        for v, _ in g[u]:      # ignore weights
            if v not in vis:
                vis.add(v)
                parent[v] = u
                q.append(v)

    # reconstruct path
    path = []
    cur = goal
    while cur:
        path.append(cur)
        cur = parent[cur]
    path.reverse()

    return path
bfs_path = bfs_unweighted(g, 'A', 'D')

In [17]:
print("\n--- BFS vs UCS COMPARISON (Unweighted Case) ---")

print("BFS Path       :", " → ".join(bfs_path))
print("BFS Cost       :", len(bfs_path) - 1)

print("UCS Path       :", " → ".join(['A','B','C','D']))
print("UCS Cost       :", 4)

print("\nObservation:")
print("Since all edges are treated as cost = 1 in BFS,")
print("BFS and UCS give the same optimal path.")



--- BFS vs UCS COMPARISON (Unweighted Case) ---
BFS Path       : A → B → D
BFS Cost       : 2
UCS Path       : A → B → C → D
UCS Cost       : 4

Observation:
Since all edges are treated as cost = 1 in BFS,
BFS and UCS give the same optimal path.


### A* ALGORITHM  

In [8]:
start = (1, 2, 3,
         4, 0, 6,
         7, 5, 8)

goal = (1, 2, 3,
        4, 5, 6,
        7, 8, 0)


In [ ]:
def h1(s): # Misplaced Tiles 
    return sum(1 for i in range(9) if s[i] != 0 and s[i] != goal[i])


In [ ]:
def h2(s): # Manhattan distance 
    d = 0
    for i in range(9):
        if s[i] != 0:
            gi = goal.index(s[i])
            d += abs(i//3 - gi//3) + abs(i%3 - gi%3)
    return d


In [11]:
def nbrs(s):
    i = s.index(0)
    r, c = i//3, i%3
    res = []

    for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
        nr, nc = r + dr, c + dc
        if 0 <= nr < 3 and 0 <= nc < 3:
            ni = nr*3 + nc
            lst = list(s)
            lst[i], lst[ni] = lst[ni], lst[i]
            res.append(tuple(lst))
    return res


In [12]:
import heapq

def astar(start, h):
    pq = []
    heapq.heappush(pq, (h(start), 0, start))  # (f, g, state)

    g = {start: 0}
    parent = {start: None}
    explored = 0

    print("\nStarting A* Search")
    print("Initial State:", start)
    print("Goal State   :", goal)
    print("-" * 50)

    while pq:
        f, cost, u = heapq.heappop(pq)
        explored += 1

        print("Expanding State:")
        print(u[0:3])
        print(u[3:6])
        print(u[6:9])
        print(f"g(n) = {cost}, h(n) = {h(u)}, f(n) = {f}")
        print("-" * 50)

        if u == goal:
            print("Goal Reached!")
            break

        for v in nbrs(u):
            nc = cost + 1
            if v not in g or nc < g[v]:
                g[v] = nc
                parent[v] = u
                heapq.heappush(pq, (nc + h(v), nc, v))
                print("  Adding Neighbor with f =", nc + h(v))

    print("\nSearch Completed")
    print("Total Nodes Expanded:", explored)
    print("Solution Depth     :", g[goal])

    return g[goal], explored


In [13]:
print("\n--- Using H1: Misplaced Tiles ---")
d1, n1 = astar(start, h1)

print("\n--- Using H2: Manhattan Distance ---")
d2, n2 = astar(start, h2)



--- Using H1: Misplaced Tiles ---

Starting A* Search
Initial State: (1, 2, 3, 4, 0, 6, 7, 5, 8)
Goal State   : (1, 2, 3, 4, 5, 6, 7, 8, 0)
--------------------------------------------------
Expanding State:
(1, 2, 3)
(4, 0, 6)
(7, 5, 8)
g(n) = 0, h(n) = 2, f(n) = 2
--------------------------------------------------
  Adding Neighbor with f = 4
  Adding Neighbor with f = 2
  Adding Neighbor with f = 4
  Adding Neighbor with f = 4
Expanding State:
(1, 2, 3)
(4, 5, 6)
(7, 0, 8)
g(n) = 1, h(n) = 1, f(n) = 2
--------------------------------------------------
  Adding Neighbor with f = 4
  Adding Neighbor with f = 2
Expanding State:
(1, 2, 3)
(4, 5, 6)
(7, 8, 0)
g(n) = 2, h(n) = 0, f(n) = 2
--------------------------------------------------
Goal Reached!

Search Completed
Total Nodes Expanded: 3
Solution Depth     : 2

--- Using H2: Manhattan Distance ---

Starting A* Search
Initial State: (1, 2, 3, 4, 0, 6, 7, 5, 8)
Goal State   : (1, 2, 3, 4, 5, 6, 7, 8, 0)
------------------------------

In [14]:
print("A* RESULTS SUMMARY")
print("------------------")

print("Using H1 (Misplaced Tiles)")
print("Solution Depth :", d1)
print("Nodes Expanded :", n1)

print("\nUsing H2 (Manhattan Distance)")
print("Solution Depth :", d2)
print("Nodes Expanded :", n2)

print("\nConclusion:")
if n2 < n1:
    print("Manhattan Distance heuristic is more efficient")
else:
    print("Both heuristics perform similarly")


A* RESULTS SUMMARY
------------------
Using H1 (Misplaced Tiles)
Solution Depth : 2
Nodes Expanded : 3

Using H2 (Manhattan Distance)
Solution Depth : 2
Nodes Expanded : 3

Conclusion:
Both heuristics perform similarly
